## News scraping

This notebook exctracts news from www.minuto30.com based on the tutorial in https://www.dataquest.io/blog/web-scraping-tutorial-python/

Get the html code for the headlines page for Medellín

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

page_med = requests.get("https://www.minuto30.com/medellin/")
soup_med = BeautifulSoup(page_med.content, 'html.parser')

The page presents all the artciles in a container of class "main-content". Within that container, news articles are in boxes of the class "mh-posts-list-content".

We extract the code in the main container and then find all the articles. We display the code for the first article.

In [3]:
news = soup_med.find(id="main-content")
articles = news.find_all(class_="mh-posts-list-content")
ar = articles[0]
print(ar.prettify())

<div class="mh-posts-list-content clearfix">
 <header class="mh-posts-list-header">
  <h3 class="entry-title mh-posts-list-title">
   <a href="https://www.minuto30.com/caminaba-por-el-barrio-cristobal-cuando-motorizados-se-acercaron-y-dispararon-en-su-contra/635435/" rel="bookmark" title="Caminaba por el barrio Cristóbal cuando motorizados se acercaron y dispararon en su contra">
    Caminaba por el barrio Cristóbal cuando motorizados se acercaron y dispararon en su contra
   </a>
  </h3>
  <div class="mh-meta entry-meta">
   <span class="entry-meta-date updated">
    <i class="fa fa-clock-o">
    </i>
    <a href="https://www.minuto30.com/date/2018/06/">
     2 junio, 2018
    </a>
   </span>
   <span class="entry-meta-author author vcard">
    <i class="fa fa-user">
    </i>
    <a class="fn" href="https://www.minuto30.com/author/mauricio-berrio/">
     Mauricio Berrío
    </a>
   </span>
  </div>
 </header>
 <div class="mh-posts-list-excerpt clearfix">
  <div class="mh-excerpt">
   

We extract the title, date, and author from each article.

In [4]:
titles = news.select(".mh-posts-list-content .entry-title")
titles = [ti.get_text() for ti in titles]
titles = [re.sub("(\\n|\\t)","",ti) for ti in titles]
print(titles)

dates = news.select(".mh-meta .entry-meta-date")
dates = [d.get_text() for d in dates]
print(dates)

authors = news.select(".mh-meta .entry-meta-author")
authors = [at.get_text() for at in authors]
print(authors)

['Caminaba por el barrio Cristóbal cuando motorizados se acercaron y dispararon en su contra', 'VIDEO y FOTOS: Se le montaron a la volqueta y le dispararon en repetidas ocasiones', 'Dos carros y tres motos fueron recuperados por la Policía en el Valle de Aburrá', 'VIDEO: Una valiente patrullera, entre los policías que lograron frustrar el robo a banco en La América', 'FOTOS: Debido a una fuga en la red hay interrupción del servicio de acueducto en sectores de Medellín y Bello', 'FOTOS: Dos menores caminaban por El Danubio y fueron atacados a disparos', 'En medio de una riña un hombre resultó muerto en el Centro de Medellín', 'Ladrón fue capturado en Las Palmas con elementos avaluados en $400.000', 'EN VIDEO: Tracto camión se quedó sin frenos bajando del túnel de Occidente', 'Capturado en el barrio 13 de Noviembre']
['2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '2 junio, 2018', '1 junio, 2018', '1 junio, 2018']
['

We organize the results in a dataframe.

In [7]:
news = pd.DataFrame({
        "date": dates, 
        "author": authors, 
        "title": titles
    })
news

,author,date,title
0,Mauricio Berrío,"2 junio, 2018",Caminaba por el barrio Cristóbal cuando motori...
1,Mauricio Berrío,"2 junio, 2018",VIDEO y FOTOS: Se le montaron a la volqueta y ...
2,Mauricio Berrío,"2 junio, 2018",Dos carros y tres motos fueron recuperados por...
3,ZPaola Agredo,"2 junio, 2018","VIDEO: Una valiente patrullera, entre los poli..."
4,ZPaola Agredo,"2 junio, 2018",FOTOS: Debido a una fuga en la red hay interru...
5,Mauricio Berrío,"2 junio, 2018",FOTOS: Dos menores caminaban por El Danubio y ...
6,Mauricio Berrío,"2 junio, 2018",En medio de una riña un hombre resultó muerto ...
7,Mauricio Berrío,"2 junio, 2018",Ladrón fue capturado en Las Palmas con element...
8,Stephany Ceballos,"1 junio, 2018",EN VIDEO: Tracto camión se quedó sin frenos ba...
9,Mauricio Berrío,"1 junio, 2018",Capturado en el barrio 13 de Noviembre
